In [117]:
import matplotlib.pyplot as plt 
import os 
import numpy as np 
import torch 
import glob 
import torchvision 
import torchvision.transforms as transforms # transform data
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim # optimzer
import pathlib 
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
import math
import random
from PIL import Image, ImageOps, ImageEnhance
import numbers


In [118]:
train_path= 'MINIST\\train\\'
test_path='MINIST\\test\\'

In [119]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device='cpu'
print(device)

cpu


In [120]:
class RandomRotation(object):

    def __init__(self, degrees, resample=False, expand=False, center=None):
        if isinstance(degrees, numbers.Number):
            if degrees < 0:
                raise ValueError("If degrees is a single number, it must be positive.")
            self.degrees = (-degrees, degrees)
        else:
            if len(degrees) != 2:
                raise ValueError("If degrees is a sequence, it must be of len 2.")
            self.degrees = degrees

        self.resample = resample
        self.expand = expand
        self.center = center

    @staticmethod
    def get_params(degrees):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        angle = np.random.uniform(degrees[0], degrees[1])

        return angle

    def __call__(self, img):
        """
            img (PIL Image): Image to be rotated.
        Returns:
            PIL Image: Rotated image.
        """
        
        def rotate(img, angle, resample=False, expand=False, center=None):
            """Rotate the image by angle and then (optionally) translate it by (n_columns, n_rows)
            Args:
            img (PIL Image): PIL Image to be rotated.
            angle ({float, int}): In degrees degrees counter clockwise order.
            resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
            expand (bool, optional): Optional expansion flag.
            If true, expands the output image to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
            center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
            """
                
            return img.rotate(angle, resample, expand, center)

        angle = self.get_params(self.degrees)

        return rotate(img, angle, self.resample, self.expand, self.center)


In [121]:
class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift
        
    @staticmethod
    def get_params(shift):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        hshift, vshift = np.random.uniform(-shift, shift, size=2)

        return hshift, vshift 
    def __call__(self, img):
        hshift, vshift = self.get_params(self.shift)
        
        return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)
    

In [122]:

transform = transforms.Compose([#transforms.ToPILImage(),
                                RandomRotation(degrees=20), RandomShift(3),
                                transforms.ToTensor(), 
                                transforms.Normalize(mean=(0.5,), std=(0.5,))
                            ])


transform_test = transforms.Compose([ #transforms.ToPILImage(),
                                transforms.ToTensor(), 
                                transforms.Normalize(mean=(0.5,), std=(0.5,))
                            ])
    
batch_size = 32
num_workers = 2

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path , transform=transform ) , 
    batch_size=batch_size , shuffle=True 
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path , transform=transform_test ) , 
    batch_size=batch_size , shuffle=False 
)
# https://www.kaggle.com/code/juiyangchang/cnn-with-pytorch-0-995-accuracy/notebook

In [123]:

def imshow(img):
  ''' function to show image '''
  # img = img / 2 + 0.5 # unnormalize
  npimg = img.numpy() # convert to numpy objects
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

# for i, (images , labels ) in enumerate(train_loader):
#         if(i < 3) :
#           imshow(torchvision.utils.make_grid(images))        

# # get random training images with iter function
# dataiter = iter(train_loader)
# images, labels = next(dataiter)

In [124]:

root=pathlib.Path(train_path)
classes=sorted([j.name.split('/') [-1] for j in root.iterdir()])
print(classes)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [164]:
class CNN(nn.Module):    
    def __init__(self,num_classes=5):
        super(CNN,self).__init__()
        #input shape : (batchsize , num of channels , height ,width )
        #((w-f+2P)/s) +1 
        self.conv1  = nn.Conv2d(3 , 32 , kernel_size=3, stride=1, padding=1),
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.relu1 = nn.ReLU()

        self.conv2  = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(32),
        self.relu2 = nn.ReLU(inplace=True), 

        self.pool1 = nn.MaxPool2d(kernel_size=2)#     nn.MaxPool2d(kernel_size=2, stride=2),
        
    
        self.conv3  = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.relu3 = nn.ReLU(), #     nn.ReLU(inplace=True),

        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1),
        self.bn4 = nn.BatchNorm2d(64),
        self.relu4 = nn.ReLU(inplace=True),

        self.pool2=  nn.MaxPool2d(kernel_size=2, stride=2) 

        self.fc = nn.Linear(64*75*75 , out_features=num_classes)


    def forward(self , input ) : 
        output= self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output= self.conv2(output)
        output=self.relu2(output)

        output=self.pool1(output)

        output= self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        output=self.pool2(output)
        # out put shape with be (256,32,75,75 )
        
        output=output.view(-1,64*4*4)
        output=self.fc(output)

        return output

    


# #cnn network

# class CNN(nn.Module) :
#     def __init__(self,num_classes=5):
#         super(CNN,self).__init__()
#         #input shape : (batchsize , num of channels , height ,width )
#         #((w-f+2P)/s) +1 
#         self.conv1  = nn.Conv2d(in_channels=3,out_channels=20,kernel_size=3,stride=1,padding=1)
#         #shape = (256 , 20 , 150 , 150 )
#         self.bn1 = nn.BatchNorm2d(num_features=20)
#         self.relu1 = nn.ReLU()

#         self.pool = nn.MaxPool2d(kernel_size=2)
#         #reduce image size by a factor of 2 
#         #shape= (256,12,75,75)

#         self.conv2  = nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
#         #shape = (256 , 32, 75 , 75 )
#         # self.bn1 = nn.BatchNorm2d(num_features=12)
#         self.relu2 = nn.ReLU()
#         #shape = (256 , 32 , 75 , 75 )

#         self.conv3  = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
#         #shape = (256 , 64 , 75 , 75 )
#         self.bn3 = nn.BatchNorm2d(num_features=64)
#         #shape = (256 , 64 , 75 , 75 )
#         self.relu3 = nn.ReLU()

#         self.fc = nn.Linear(64*75*75 , out_features=num_classes)


#     def forward(self , input ) : 
#         output= self.conv1(input)
#         output=self.bn1(output)
#         output=self.relu1(output)

#         output=self.pool(output)

#         output= self.conv2(output)
#         output=self.relu2(output)

#         output= self.conv3(output)
#         output=self.bn3(output)
#         output=self.relu3(output)
#         # out put shape with be (256,32,75,75 )
        
#         output=output.view(-1,64*75*75)
#         output=self.fc(output)

#         return output


        
        

In [165]:
model = CNN()
optimizer =Adam(model.parameters( ) , lr=0.0001 )
loss_function = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [149]:
epochs=50

In [166]:
train_cnt = len(glob.glob(train_path+'/**/*.jpg') )
test_cnt = len(glob.glob(test_path+'/**/*.jpg') )
print(train_cnt , test_cnt)

7291 2007


In [167]:
def train(epoch):
    model.train()
    for i, (images , labels ) in enumerate(train_loader):
        images, labels = Variable(images), Variable(labels)
        if torch.cuda.is_available() :
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        # print( images.shape)
        exp_lr_scheduler.step()
        outputs = model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
    if (i + 1)% 100 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, (i + 1) * len(images), len(train_loader.dataset),
            100. * (i + 1) / len(train_loader), loss.data[0]))


def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:   
            data, target = Variable(data, volatile=True), Variable(target)
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
            
            output = model(data)
            
            loss += F.cross_entropy(output, target, size_average=False).data

            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
    



In [169]:
n_epochs = 1
for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

TypeError: 'tuple' object is not callable

In [ ]:
def prediciton(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    for i, data in enumerate(data_loader):
        data = Variable(data, volatile=True)
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred

In [144]:
test_pred = prediciton(test_loader)
evaluate(test_loader)
# out_df = pd.DataFrame(np.c_[np.arange(1, len(test_dataset)+1)[:,None], test_pred.numpy()], 
#                       columns=['ImageId', 'Label'])

NameError: name 'prediciton' is not defined